활용데이터 - https://www.kaggle.com/datasets/aliiihussain/amazon-sales-dataset

In [20]:
# 프로젝트 루트 경로 유지
from pathlib import Path

BASE_DIR = Path.cwd()
if BASE_DIR.name == "notebooks":
    BASE_DIR = BASE_DIR.parent

In [21]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
import statsmodels.api as sm

In [22]:
df = pd.read_csv(BASE_DIR/"data"/"amazon_sales_dataset.csv")

In [23]:
df

,order_id,order_date,product_id,product_category,price,discount_percent,quantity_sold,customer_region,payment_method,rating,review_count,discounted_price,total_revenue
0,1,2022-04-13,2637,Books,128.75,10,4,North America,UPI,3.5,443,115.88,463.52
1,2,2023-03-12,2300,Fashion,302.60,20,5,Asia,Credit Card,3.7,475,242.08,1210.40
2,3,2022-09-28,3670,Sports,495.80,20,2,Europe,UPI,4.4,183,396.64,793.28
3,4,2022-04-17,2522,Books,371.95,15,4,Middle East,UPI,5.0,212,316.16,1264.64
4,5,2022-03-13,1717,Beauty,201.68,0,4,Middle East,UPI,4.6,308,201.68,806.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49996,2022-09-03,1433,Beauty,26.99,0,5,Middle East,Credit Card,2.4,386,26.99,134.95
49996,49997,2022-07-03,1428,Beauty,294.23,10,5,Asia,Credit Card,3.1,8,264.81,1324.05
49997,49998,2023-02-17,4651,Electronics,352.11,30,4,Asia,Debit Card,3.1,104,246.48,985.92
49998,49999,2022-09-30,4371,Beauty,307.54,5,1,Middle East,UPI,1.8,316,292.16,292.16


## 1.매출량에 할인이 영향이 있는가?

In [24]:
df['discount_percent'] = df['discount_percent']/100

In [25]:
df['order_date'] = pd.to_datetime(df['order_date'])

In [26]:
df['year']= df['order_date'].dt.year

In [27]:
category_year = df.groupby(['product_category','year']).agg(
    주문량 = ('order_id','count'),
    판매금액 = ('total_revenue','sum'),
    평균할인율 = ('discount_percent','mean')
).reset_index()

In [28]:
category_year = category_year.rename(columns={'product_category':'카테고리'})

In [29]:
category_year

,카테고리,year,주문량,판매금액,평균할인율
0,Beauty,2022,4175,2736561.90,0.133760
1,Beauty,2023,4290,2814063.07,0.133671
2,Books,2022,4159,2719289.95,0.133253
3,Books,2023,4168,2765573.08,0.133493
4,Electronics,2022,4131,2724266.75,0.134193
5,Electronics,2023,4189,2746327.28,0.130998
6,Fashion,2022,4153,2754490.36,0.132434
7,Fashion,2023,4212,2725632.98,0.134722
8,Home & Kitchen,2022,4116,2732327.71,0.133467
9,Home & Kitchen,2023,4142,2740804.84,0.132762


In [30]:
pivot = category_year.pivot(index='카테고리', columns='year')

pivot['주문 성장률'] = round((
    (pivot['주문량'][2023] - pivot['주문량'][2022])/ pivot['주문량'][2022] * 100),2)

pivot['매출 성장률'] = round((
    (pivot['판매금액'][2023] - pivot['판매금액'][2022])/ pivot['판매금액'][2022] * 100),2)

pivot['할인율 변화'] = (pivot['평균할인율'][2023]- pivot['평균할인율'][2022]) * 100

result = pivot[['주문 성장률','매출 성장률','할인율 변화']].reset_index()

In [31]:
result

,카테고리,주문 성장률,매출 성장률,할인율 변화
year,,,,
0,Beauty,2.75,2.83,-0.008915
1,Books,0.22,1.70,0.024010
2,Electronics,1.40,0.81,-0.319484
3,Fashion,1.42,-1.05,0.228784
4,Home & Kitchen,0.63,0.31,-0.070501
5,Sports,-2.84,-1.38,-0.062838


In [32]:
#스피어만 상관계수
corr_orders = spearmanr(result['할인율 변화'], result['주문 성장률'])
corr_revenue = spearmanr(result['할인율 변화'], result['매출 성장률'])

In [33]:
print(corr_orders)
print(corr_revenue)

SignificanceResult(statistic=np.float64(0.2), pvalue=np.float64(0.704))
SignificanceResult(statistic=np.float64(0.028571428571428574), pvalue=np.float64(0.9571545189504373))


##### 판매량 성장에 할인의 영향  
Spearman = 0.2 -> 약한 상관 관계 // p-value(0.704) > 0.05 -> 유의X  
판매량에 할인은 영향을 끼치지 않는다. 

##### 매출 성장에 할인의 영향  
Spearman = 0.028 -> 관계 X // p-value(0.957) > 0.05 -> 유의X  
매출에 할인은 영향을 끼치지 않는다.  

## 2.카테고리별로 매출 성장 구조 분석

In [34]:
df['year'] = pd.to_datetime(df['order_date']).dt.year
#AOV = 평균 주문금액
cat_year = df.groupby(['product_category','year']).agg(
    판매량=('total_revenue','sum'),
    주문량=('order_id','count')
).reset_index()

cat_year['객단가'] = cat_year['판매량'] / cat_year['주문량']
cat_year = cat_year.rename(columns={'product_category':'카테고리'})

In [35]:
pivot_1 = cat_year.pivot(index='카테고리', columns='year')

pivot_1['주문 성장률'] = round((
    (pivot_1['주문량'][2023] - pivot_1['주문량'][2022]) / pivot_1['주문량'][2022] * 100),2)
pivot_1['매출 성장률'] = round((
    (pivot_1['판매량'][2023] - pivot_1['판매량'][2022]) / pivot_1['판매량'][2022] * 100),2)

pivot_1['객단가 차이'] = (pivot_1['객단가'][2023] - pivot_1['객단가'][2022]) / pivot_1['객단가'][2022] * 100

result_1 = pivot_1[['주문 성장률','매출 성장률','객단가 차이']].reset_index()

In [36]:
result_1

,카테고리,주문 성장률,매출 성장률,객단가 차이
year,,,,
0,Beauty,2.75,2.83,0.075493
1,Books,0.22,1.70,1.482424
2,Electronics,1.40,0.81,-0.586012
3,Fashion,1.42,-1.05,-2.433733
4,Home & Kitchen,0.63,0.31,-0.319411
5,Sports,-2.84,-1.38,1.496449


Beauty : 매출 +2.83% / 주문: +2.75% / 주문금액율 : 0.07%     
--> 주문량 증가로 인한 매출 상승    
Books : 매출 +1.7% / 주문: +0.22% / 주문금액율 : 1.48%   
--> 주문량 변화가 별로 없지만 주문 금액 상승으로 인한 매출 상승    
Electronics : 매출 +0.81% / 주문 +1.40% / 주문금액율 -0.59%   
--> 평균 주문 금액이 감소했지만 주문량 증가로 인한 매출 상승   
Fashion : 매출 -1.05% / 주문 +1.42% / 주문금액율 -2.43%   
--> 주문량은 증가했지만 평균 주문 금액 감소로 매출 하락    
Home & Kitchen : 매출 +0.31% / 주문 +0.63% / 주문금액율 -0.32%   
--> 팽균 주문 금액이 감소했지만 주문량 증가로 인한매출 상승    
Sports : 매출 -1.38% / 주문 -2.84% / 주문금액율 +1.50%  
--> 평균 주문 금액은 상승했지만 주문량 감소로 인한 매출 하락  